## Libraries importation

In [ ]:
# Please use these lines in every notebook you create

import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from global_variables import *

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import Delaunay, Voronoi
from copy import deepcopy
import math

from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.neighbours_criteria.enhanced_criteria import distance_criterion_enhanced
from python_scripts.city.city_utils import mean_distance_to_NN
from python_scripts.miscellaneaous.data_processing import extract_data

## Database import and data extraction

In [ ]:
df_extracted = pd.read_csv("../../database/short-Normandy-antenn.csv", sep=";", decimal=".")
df_extracted.head()

In [ ]:
df2 = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df3 = extract_data(df2, provider=PROVIDER, region=REGION, techno=TECHNO)
df3.head()

## City detection

In [ ]:
mean_distances = mean_distance_to_NN(df3[['x', 'y']], n_neighbours=N_NEIGH)

## Voronoi and Delaunay creation

In [ ]:
points_pos = df_extracted[['latitude', 'longitude']].values

In [ ]:
# Perform Voronoi diagram
voronoi = Voronoi(points_pos)

In [ ]:
# Perform Delaunay triangulation
delaunay = Delaunay(points_pos)

In [ ]:
# Function to calculate the angle between two points
def calculate_angle(pt1, pt2):
    angle = np.degrees(np.arctan2(pt2[1] - pt1[1], pt2[0] - pt1[0]))
    return ((angle + 360) % 360) - 90

In [ ]:
# Function to check if angle is within coverage
def is_within_coverage(station_pos, neighbor_pos, azimuth, beamwidth):
    direction_vector = neighbor_pos - station_pos
    direction_angle = (np.degrees(np.arctan2(direction_vector[1], direction_vector[0])) + 360) % 360
    min_angle = (azimuth - beamwidth / 2 + 360) % 360
    max_angle = (azimuth + beamwidth / 2 + 360) % 360

    if min_angle <= max_angle:
        return min_angle <= direction_angle <= max_angle
    else:
        return direction_angle >= min_angle or direction_angle <= max_angle

    

In [ ]:
# Function to find real neighbors based on Delaunay triangulation and azimuths
def find_real_neighbors(df_azimuth, df):
    del_G, pos = delaunay_graph(df)
    neigh_G = deepcopy(del_G)
    
    for bs_id in df.index:
        station_row = df.loc[bs_id]
        unique_azimuths = np.unique(df_azimuth.loc[df_azimuth['id_station_anfr'] == station_row['id_station_anfr'], 'AER_NB_AZIMUT'])
        nb_azimuth = len(unique_azimuths)
        beamwidth = 360 / nb_azimuth if nb_azimuth > 1 else 360
        
        for neigh_id in [neigh for [bs_id, neigh] in del_G.edges(bs_id)]:
            neighbor_row = df.loc[neigh_id]
            is_neighbor = False

            for azimuth in unique_azimuths:
                if is_within_coverage(pos[bs_id], pos[neigh_id], azimuth, beamwidth):
                    unique_neigh_azimuth = np.unique(df_azimuth.loc[df_azimuth['id_station_anfr'] == neighbor_row['id_station_anfr'], 'AER_NB_AZIMUT'])
                    neigh_nb_azimuth = len(unique_neigh_azimuth)
                    beamwidth_neigh = 360 / neigh_nb_azimuth

                    for neighbor_azimuth in unique_neigh_azimuth:
                        if is_within_coverage(pos[neigh_id], pos[bs_id], neighbor_azimuth, beamwidth_neigh):
                            is_neighbor = True
                            break

            if not is_neighbor:
                neigh_G.remove_edges_from([[bs_id, neigh_id]])

    return neigh_G


In [ ]:
neigh_G = find_real_neighbors(df_extracted, df3)

In [ ]:
neigh_G.edges(29)

## Map creation

In [ ]:
import folium
import numpy as np
from networkx import Graph
from pandas import DataFrame, Series

from python_scripts.city.city_utils import mean_distance_choice

In [ ]:
def add_graph_edges(G_base: Graph, G: Graph, df: DataFrame, fg: folium.FeatureGroup, colour: str):
    for edge in G_base.edges:
        stations = []

        if(not(edge in G.edges)):
            stations.append(df.loc[edge[0], ['latitude', 'longitude']])
            stations.append(df.loc[edge[1], ['latitude', 'longitude']])

            folium.PolyLine(np.array(stations), color=colour, weight=2.5, opacity=1).add_to(fg)

In [ ]:
## Antennas visualization
def add_azimuth_lines(df: DataFrame, fg: folium.FeatureGroup, length):
    for ind, row in df.iterrows():
        station_data = df[df['id_station_anfr'] == row['id_station_anfr']]
        for idx, antenna in station_data.iterrows():
            azimuth_angle = np.radians(antenna['AER_NB_AZIMUT'])
            end_lat = row['latitude'] + length * np.cos(azimuth_angle)
            end_lon = row['longitude'] + length * np.sin(azimuth_angle)
            folium.PolyLine([(row['latitude'], row['longitude']), (end_lat, end_lon)], color='black', weight=2, opacity=0.7).add_to(fg)

## Map visualisation
def create_method_illustation_map(df: DataFrame, df_azimuth, del_graph: Graph, nei_graph: Graph, save_as: str, **kwargs):
    map = folium.Map(location=list(np.mean(df[['latitude','longitude']], axis=0)), zoom_start=8.5, tiles="Cartodb Positron")

    edges_del = folium.FeatureGroup(f"Edges - Delaunay triangulation ({len(del_graph.edges)})", show=False).add_to(map)
    edges_nei = folium.FeatureGroup(f"Edges - neighbouring graph ({len(nei_graph.edges)})", show=False).add_to(map)
    azimuth_lines = folium.FeatureGroup(f"Azimuth Lines", show=False).add_to(map)

    add_graph_edges(del_graph, Graph(), df, edges_del, colour="lightblue")
    add_graph_edges(nei_graph, Graph(), df, edges_nei, colour="#AAA662")
    azimuth_length = 0.01
    add_azimuth_lines(df_azimuth, azimuth_lines, azimuth_length)

    points = folium.FeatureGroup(f"Base stations ({len(df)})").add_to(map)

    for ind, row in df.iterrows():
        station_data = df_azimuth[df_azimuth['id_station_anfr'] == row['id_station_anfr']]
        popup_text = (
            f"Station ID: {row['id_station_anfr']}<br>"
            f"Department: {row['nom_dep']}<br>"
            f"Commune: {row['nom_com']}<br>"
            f"Coordinates: ({row['latitude']}, {row['longitude']})<br>"
            f"2G: {row['site_2g']}<br>"
            f"3G: {row['site_3g']}<br>"
            # f"4G: {row['site_4g']}<br>"
            f"5G: {row['site_5g']}<br>"
        )
        for idx, antenna in station_data.iterrows():
            popup_text += (
                f"<br>Antenna Dimension: {antenna['AER_NB_DIMENSION']}<br>"
                f"Antenna Azimuth: {antenna['AER_NB_AZIMUT']}<br>"
            )
        popup = folium.Popup(popup_text, max_width=150)
        folium.CircleMarker(location=[row['latitude'], row['longitude']], color='blue', radius=3, popup=popup, fillOpacity=1, fill=True).add_to(points)

    folium.LayerControl().add_to(map)

    map.save(f"../../out/maps/neighbours_finding/{save_as}.html")

In [ ]:
del_G, pos = delaunay_graph(df3)
# neigh_G = distance_criterion_enhanced(neigh_G, pos, params=MEAN_DISTANCE_PARAMS, mean_distance_to_NN=mean_distances)
create_method_illustation_map(df3, df_extracted, del_G, neigh_G, save_as="anatoli_tmp")

Adding aditional visualization